### Agentic AI - Prompts and Documents

In [ ]:
import os

from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger, Document
from termcolor import colored

In [ ]:
load_dotenv()

In [ ]:
if 'TAVILY_SEARCH_API_KEY' not in os.environ:
    print(colored("Warning: TAVILY_SEARCH_API_KEY is not in OS Environment Variables, Please set!", "yellow"))

In [ ]:
# base_url = f"http://{os.environ["LLAMA_STACK_SERVER_REMOTE_HOST"]}:{os.environ["LLAMA_STACK_SERVER_REMOTE_PORT"]}"

# client = LlamaStackClient(
#     base_url=base_url,
#     provider_data={
#         "tavily_search_api_key": os.environ["TAVILY_SEARCH_API_KEY"]
#     }
# )

host = os.environ["TOGETHER_URL"]
together_api_key = os.environ["TOGETHER_API_KEY"]

client = LlamaStackClient(
    base_url=host,
    provider_data={
        "tavily_search_api_key": os.environ["TAVILY_SEARCH_API_KEY"],
        "together_api_key": together_api_key
    }
)

In [ ]:
available_shields = [shield.identifier for shield in client.shields.list()]

if not available_shields:
    print(colored("No Shields are available", "yellow"))
else:
    print(f"Available Sheilds: {available_shields}")

In [ ]:
available_models = [model.identifier for model in client.models.list()]

print(f"Available Models: {available_models}")

In [ ]:
selected_model = "meta-llama/Llama-3.2-3B-Instruct"

In [ ]:
print(colored(f"Selected Model: {selected_model}", "yellow"))

In [ ]:
tools = client.tools.list()

tools

In [ ]:
agent = Agent(
    client,
    model=selected_model,
    instructions="You're a helpful assistant, who answers questions professionally",
    sampling_params={
        "temperature": 0.2,
        "type": "top_p",
        "top_p": 0.9,
        "max_tokens": 1000
    },
    tools=["builtin::websearch", "builtin::code_interpreter"],
    input_shields=[],
    output_shields=[],
    enable_session_persistence=False
)

In [ ]:
session_id = agent.create_session("test-session")

print(f"Session Created {session_id} with the agent {agent.agent_id}")

In [ ]:
with open('./transcript.txt', 'r') as file:
    data = file.read().rstrip()

In [ ]:
user_prompts = [
    (
        "Here is the podcast transcript as attachment, can you summarize it!",
        [
            Document(
                content = data,
                mime_type="text/plain"
            )
        ]
    ),
    (
        "what are the top 3 topics that they were discussing", None
    ),
    (
        "Was anything related to H100 machines discussed?", None
    ),
    (
        """while this podcast happened in April 2024, can you search and provide an updated information from the web
            on what were the key developments that have happend since then, in the last 3 months
        """, None
    ),
    (
        """
        Image these people meet again in 1 year, what might there be three good follow-ups to discuss
        """, None
    ),
    (
        "Hey, can you write these follow-ups in Hindi?", None
    )
]

In [ ]:
for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt[0]
            }
        ],
        documents=prompt[1],
        session_id=session_id
    )

    for log in AgentEventLogger().log(response):
        log.print()